Notebook для практики по дисциплине "Методы машинного обучения"
Выполнили студенты:
- Баймухамедов Рафаэль Русланович
- Аршин Александр Дмитриевич
- Пасечный Леонид Витальевич

Преподаватель
- Петруша Полина Георгиевна

Скачаем датасет с Яндекс.Диска

In [1]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = "https://disk.yandex.ru/d/V1sJpR-SUJ_b8A"

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

download_response = requests.get(download_url)
with open('dataset.xlsx', 'wb') as f:
    f.write(download_response.content)

Прочитаем в датафрейм наш файл

In [2]:
from pathlib import Path
import pandas as pd

xlsx_path = "dataset.xlsx"
if not xlsx_path:
    raise FileNotFoundError("xlsx файл не найден")
print("Найден XLSX:", xlsx_path)

df = pd.read_excel(xlsx_path, sheet_name=0, header=[0,1])
print("Данные загружены в df")

Найден XLSX: dataset.xlsx
Данные загружены в df


Настроим pandas

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

Посмотрим на него

In [4]:
df.head()

Unnamed: 0_level_0  \
                                                                                                                            Наименование   
0                                                    биологически активная добавка к пище "ВитаФин" ("VitaFin") (таблетки массой 700 мг)   
1                                               биологически активная добавка к пище "Гепаплант" ("Hepaplant") (таблетки массой 1000 мг)   
2  биологически активная добавка к пище "Фемибион ® Наталкер I" ("Femibion® Natalcare I") (таблетки массой 609 мг±3% покрытые оболочкой)   
3                                 биологически активная добавка к пище "Неогален (Neogalen). СЛИМ КОД. Комплекс" (капсулы массой 500 мг)   
4                                                                   биологически активная добавка к пище "КРЕСТИНОЛ" (капсулы по 350 мг)   

                                                                                                                                                                                                                                                                                                                      Unnamed: 1_level_0  \
                                                                                                                                                                                                                                                                                                                            Изготовитель   
0                                                                                                                                                                                                                                                    Ханкинтатукку Оу" ("Hankintatukku Oy"), Lehtolankatu 18, 03600 Karkkila (Финляндия)   
1                                                                                                                                                                                                                                                    Ханкинтатукку Оу" ("Hankintatukku Oy"), Lehtolankatu 18, 03600 Karkkila (Финляндия)   
2  "Мерк КгаА энд Ко Верк Шпитталь", Хёссельгассе 20, А- 9800 Шпитталь/Драу, Австрия или "Никомед Фарма Сп. з.о.о.", Ал. Иерозолимские 146 А, 02-305, Варшава, Польша под контролем "Мерк Зельбстмедикацион ГмбХ, Pёсслерштрассе, 96, 64293 Дармштадт ("Merck Selbstmedikation GmbH", Rosslerstrasse 96, 64293 Darmstadt) (ФРГ) Германия   
3                                                                                                                                  продукция изготовлена ООО "КоролёвФарм", 121609, г. Москва, ул. Крылатские холмы, д.1, кв.422 (адрес производства: 141070, Московская область, г.Королёв, ул. Пионерская, д.4) (Российская Федерация)   
4                                                                                                                                                                                                                                                   ЗАО "Пекталь", 193019, г. Санкт-Петербург, ул. Бехтерева, д.1 (Российская Федерация)   

                              Unnamed: 2_level_0  \
                      Номер свидетельства и дата   
0   RU.77.99.11.003.Е.000894.09.10 от 10.09.2010   
1   RU.77.99.11.003.Е.000895.09.10 от 10.09.2010   
2  RU.77.99.11.003.Е.000886.09.10 от 10.09.2010    
3   RU.77.99.11.003.Е.001040.09.10 от 14.09.2010   
4   RU.77.99.11.003.Е.001573.10.10 от 05.10.2010   

                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [5]:
import re
from collections import Counter, defaultdict

def clean(s):
    if s is None: return ""
    s = str(s).replace("\n"," ").replace("\xa0"," ").strip()
    return re.sub(r"\s+"," ", s)

flat = []
for top, sub in df.columns:
    top, sub = clean(top), clean(sub)
    name = sub if (not top or top.lower().startswith("unnamed")) else f"{top}__{sub}" if sub else top
    name = name.replace("ё","е")
    name = re.sub(r"\s+", "_", name)
    name = re.sub(r"[\\/:;,\"'()]+", "_", name)
    name = re.sub(r"_+", "_", name).strip("_")
    name = name.lower()
    flat.append(name)

cnt = Counter(flat); seen = defaultdict(int); uniq = []
for n in flat:
    seen[n] += 1
    uniq.append(n if cnt[n] == 1 else f"{n}__{seen[n]}")

df.columns = uniq

In [6]:
to_rename = {
    "пищевые_вещества_макро-_и_микроэлементы": "пищевые_вещества_макро_и_микроэлементы",
    "минеральные_и_минерало-органические_природные_субстанции_цеолиты_гуминовые_кислоты":"минеральные_и_минерало_органические_природные_субстанции_цеолиты_и_гуминовые_кислоты",
    "система_органов_костно-мышечная_сиситема": "система_органов_костно_мышечная_система",
    "система_органов_форма_выпуска":"форма_выпуска",
    "система_органов_продолжительность_приема":"продолжительность_приема",
    "система_органов_происхождение":"происхождение",
    "система_органов_сырье_растительное_животное_биологическое":"сырье"

}

df = df.rename(columns=to_rename)

print("Имена колонок (первые 50):")
for c in df.columns[:]:
    print("-", c)

df.head()

Имена колонок (первые 50):
- наименование
- изготовитель
- номер_свидетельства_и_дата
- этикетка
- происхождение
- противопоказания
- срок_годности
- год_регистрации
- количество_групп_компонентов
- пищевые_вещества_витамины_витаминоподобные_вещества_и_коферменты
- пищевые_вещества_макро_и_микроэлементы
- пищевые_вещества_жиры_жироподобные_вещества_и_их_производные
- пищевые_вещества_белки_пептиды_аминокислоты_нуклеиновые_кислоты
- минорные_компоненты_растений_фенольные_соединения
- минорные_компоненты_растений_алкалоиды
- пробиотики_в_монокультурах_и_ассоциациях_пробиотические_микроорганизмы
- пищевые_вещества_углеводы_и_продукты_их_переработки
- минорные_компоненты_растений_сапонины
- минорные_компоненты_растений_терпеноиды
- минорные_компоненты_растений_естественные_метаболиты_и_стимуляторы_метаболизма
- минорные_компоненты_растений_гидроксикоричные_кислоты
- минорные_компоненты_растений_ферменты
- минорные_компоненты_растений_дубильные_вещества
- минеральные_и_минерало_органические

,наименование,изготовитель,номер_свидетельства_и_дата,этикетка,происхождение,противопоказания,срок_годности,год_регистрации,количество_групп_компонентов,пищевые_вещества_витамины_витаминоподобные_вещества_и_коферменты,пищевые_вещества_макро_и_микроэлементы,пищевые_вещества_жиры_жироподобные_вещества_и_их_производные,пищевые_вещества_белки_пептиды_аминокислоты_нуклеиновые_кислоты,минорные_компоненты_растений_фенольные_соединения,минорные_компоненты_растений_алкалоиды,пробиотики_в_монокультурах_и_ассоциациях_пробиотические_микроорганизмы,пищевые_вещества_углеводы_и_продукты_их_переработки,минорные_компоненты_растений_сапонины,минорные_компоненты_растений_терпеноиды,минорные_компоненты_растений_естественные_метаболиты_и_стимуляторы_метаболизма,минорные_компоненты_растений_гидроксикоричные_кислоты,минорные_компоненты_растений_ферменты,минорные_компоненты_растений_дубильные_вещества,минеральные_и_минерало_органические_природные_субстанции_цеолиты_и_гуминовые_кислоты,система_органов_для_беременных_кормящих_и_планирующих_беременность,система_органов_костно_мышечная_система,система_органов_нервная_система,система_органов_иммунная_система,система_органов_пищеварительный_тракт_и_обмен_веществ,система_органов_мочеполовая_система,система_органов_дерматологические_бад,система_органов_органы_чувств,система_органов_сердечно-сосудистая_система,система_органов_противоопухолевые_бад,система_органов_противопаразитарные_бад,система_органов_дыхательная_система,система_органов_кровь_и_система_кроветворения,система_органов_противомикробные_бад,форма_выпуска,продолжительность_приема,система_органов_происхождение_природное_синтетическое,сырье,группа_населения_предназначен_для_детей,группа_населения_возраст_детей,группа_населения_предназначен_для_взрослых,группа_населения_пол,группа_населения_пожилые
0,"биологически активная добавка к пище ""ВитаФин"" (""VitaFin"") (таблетки массой 700 мг)","Ханкинтатукку Оу"" (""Hankintatukku Oy""), Lehtolankatu 18, 03600 Karkkila (Финляндия)",RU.77.99.11.003.Е.000894.09.10 от 10.09.2010,"Рекомендации по применению: взрослым по 1-2 таблетки в день во время еды. Продолжительность приема - 1-2 месяца. При необходимости прием можно повторить. Противопоказания: индивидуальная непереносимость компонентов, беременность, кормление грудью. Перед применением рекомендуется проконсультироваться с врачом. Лицам с заболеваниями щитовидной микроэлементыы рекомендуется проконсультироваться с врачом-эндокринологом. Срок годности - 3 года. Хранить в сухом, недоступном для детей месте, при температуре не выше 25°С.",иностранное,"индивидуальная непереносимость компонентов, беременность, кормление грудью",3 года,2010,3,вит,элементы,NaN,NaN,фенольн,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,иммунитет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,таблетки,1-2 месяца,Природное (растительное),"Экстракта зелёного чая (Camellia sinensis - Камелия китайская), экстракт кожицы винограда (Винный виноград - Vitis Vinifera",NaN,NaN,взрослые,NaN,NaN
1,"биологически активная добавка к пище ""Гепаплант"" (""Hepaplant"") (таблетки массой 1000 мг)","Ханкинтатукку Оу"" (""Hankintatukku Oy""), Lehtolankatu 18, 03600 Karkkila (Финляндия)",RU.77.99.11.003.Е.000895.09.10 от 10.09.2010,"Рекомендации по применению: взрослым по 1 таблетке 2-3 раза в день во время еды. Продолжительность приема - 1-2 месяца. При необходимости прием можно повторить. Противопоказания: индивидуальная непереносимость компонентов, беременность, кормление грудью. Перед применением рекомендуется проконсультироваться с врачом. Срок годности - 3 года. Хранить в сухом, недоступном для детей месте, при температуре не выше 25°С.",иностранное,"индивидуальная непереносимость компонентов, беременность, кормление грудью",3 года,2010,много,вит,NaN,пнжк,NaN,фенольн,NaN,NaN,NaN,NaN,NaN,NaN,гидроксикор,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,таблетки,1-2 месяца,Природное (растительное),"Экстракта зелёного чая (Camellia sinensis - Камелия китайская), экстракт кожицы вино